### Importing the necesssary libraries

In [2]:
#data loading 
import scipy.io
import pandas as pd


#bases
import numpy as np

#system
import sys
import re
import dill
import os 

#scaling
from sklearn.preprocessing import StandardScaler


### Loading Data 

In [6]:
#Load fMRI Resting State Data
with open("/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Data_Sets/HCP/Resting State FMRI/fmri_rs.npy", "rb") as f:
  fmri_rs = np.load(f)

#Loading Structural FMRI data 
dMRI_streamlog = (scipy.io.loadmat("/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Data_Sets/HCP/Tactography/features_schaefer_streamcount_log.mat")['features_schaefer_streamcount_log'])

#Take the Transpose so that each Sample is a Row
fmri_rs = fmri_rs.T
dMRI_streamlog = dMRI_streamlog.T

#Get Split to divide into train + test(loaded data is in the form of features * sampels so need to transpose)
mat_file = scipy.io.loadmat("/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Data_Sets/HCP/Resting State FMRI/MMP_HCP_60_splits.mat")
seed_1 = mat_file['folds']['seed_1']
subject_lists = seed_1[0, 0]['sub_fold'][0, 0]['subject_list']
test_subjects = [int(item[0]) for item in subject_lists[0,0].flatten()]

#Get HCP test subjects
HCP_753_Subjects = []
with open('/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Data_Sets/HCP/Resting State FMRI/MMP_HCP_753_subs.txt', 'r') as file:
    HCP_753_Subjects = [int(re.sub('\n', '', line)) for line in file.readlines()]

#Put the HCP test subjects into a dataframe
df = pd.read_csv("/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Data_Sets/HCP/Resting State FMRI/MMP_HCP_componentscores.csv")
df['Subject'] = pd.to_numeric(df['Subject'], errors='coerce')
df = df[df['Subject'].isin(HCP_753_Subjects)].reset_index(drop = True)

#Split all our data into a Train and Test Set
df_train, df_test = df[~df['Subject'].isin(test_subjects)], df[df['Subject'].isin(test_subjects)]

#Create Train and Test Arrays corresponding to Training and Test Subjects
train_subjects = df_train.index.to_list()
test_subjects = df_test.index.to_list()

#Reshape Labels into Column Vectors
X_train_fmri,Y_train = fmri_rs[train_subjects], df_train["varimax_cog"].to_numpy().reshape((-1, 1))
X_test_fmri, Y_test = fmri_rs[test_subjects], df_test["varimax_cog"].to_numpy().reshape((-1, 1))

#dMRI
X_train_structural = dMRI_streamlog[train_subjects]
X_test_structural  = dMRI_streamlog[test_subjects]

In [8]:
scalar_fmri = StandardScaler()
scalar_dmri = StandardScaler()

#FMRI 

X_train_std_fmri = scalar_fmri.fit_transform(X_train_fmri)
X_test_std_fmri = scalar_fmri.transform(X_test_fmri)

#Structural

X_train_std_dmri = scalar_dmri.fit_transform(X_train_structural)
X_test_std_dmri = scalar_dmri.transform(X_test_structural)


In [10]:
for i in range(X_train_fmri.shape[0]):
    print(np.linalg.norm(X_train_std_dmri[i,:]),np.linalg.norm(X_train_std_fmri[i,:]))

222.3770533518467 269.85804593657264
257.26191944642244 230.7405031788746
235.97425396044108 244.25203730015033
268.5008458440842 287.8474812355267
226.79594439996 268.36454874072797
290.15841093584015 254.56288259964305
238.81427344566796 293.44008367194914
242.449858179746 255.84316302991843
220.24411304621657 273.38978201521695
215.04592138863524 240.68095960090898
190.3491122512485 256.3966339275528
216.53006256826592 291.5781614338442
262.5366299434632 251.31434586387934
193.98110597790088 230.73850088656434
289.65422963664804 230.79141476535293
263.25235245800695 262.1603622663756
228.51061764139695 246.04461475619735
219.9462410529965 284.63072690367113
312.3483241367462 292.94861945284157
246.16753498186176 246.88731829243187
179.67595154745803 317.83654419274234
306.62750999329893 299.2676544555967
228.73369415088 279.68988319681944
214.13104805888338 244.54664492856966
213.34873681725801 322.9809975045658
219.69089375683515 231.7772342201955
205.22348962658378 309.53673936807